In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score, explained_variance_score,
    median_absolute_error, max_error, mean_squared_log_error
)
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

import itertools

from itertools import combinations


from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS

import warnings
warnings.filterwarnings("ignore")

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# 1. Load dataset
df = pd.read_csv("carbon emissions data - Sheet4.csv")

# Identify year columns
year_cols = [c for c in df.columns if c.isdigit()]

# Reshape wide → long format
df_long = df.melt(
    id_vars=["Substance", "Sector", "EDGAR Country Code", "Country"],
    value_vars=year_cols,
    var_name="Year",
    value_name="Value"
)

df_long["Year"] = pd.to_datetime(df_long["Year"].astype(int).astype(str) + "-01-01")
df_long = df_long.sort_values(["Substance", "Sector", "Year"])


# 2. Create one time series per (Substance, Sector)
time_series_dict = {}
grouped = df_long.groupby(["Substance", "Sector"])

for (sub, sec), g in grouped:
    ts = pd.Series(g["Value"].values, index=g["Year"])
    ts.name = f"{sub}_{sec}".replace(" ", "_")
    time_series_dict[(sub, sec)] = ts


# 3. Plot and save all time series
output_folder = "/home/karan/pyKaran/CarbonEmissions/Sheet4Plots"
os.makedirs(output_folder, exist_ok=True)

for (sub, sec), ts in time_series_dict.items():
    plt.figure(figsize=(10,4))
    plt.plot(ts.index, ts.values)
    plt.title(f"{sub} - {sec}")
    plt.xlabel("Year")
    plt.ylabel("Value")
    plt.tight_layout()

    safe_name = f"{sub}_{sec}".replace(" ", "_")
    plt.savefig(f"{output_folder}/{safe_name}.png", dpi=300)
    plt.close()

print("Plots saved in:", output_folder)


Plots saved in: /home/karan/pyKaran/CarbonEmissions/Sheet4Plots


In [10]:
groups = df_long.groupby(["Substance", "Sector"])


# 1. Utility functions

def rss_linear(y, X):
    beta = np.linalg.lstsq(X, y, rcond=None)[0]
    resid = y - X @ beta
    return float(resid.T @ resid), beta

def bic_from_rss(rss, n, k):
    return n * np.log(rss / n) + k * np.log(n)

def design_matrix_with_breaks(t, breaks):
    X = np.ones((len(t), 1))
    X = np.column_stack([X, t])
    for b in breaks:
        hinge = np.maximum(0, t - b)
        X = np.column_stack([X, hinge])
    return X

def exhaustive_break_search(y, max_breaks=3, min_seg=5):
    n = len(y)
    t = np.arange(n)
    best = {"breaks": [], "bic": np.inf, "rss": None, "betas": None}

    candidates = list(range(min_seg, n - min_seg + 1))

    X0 = design_matrix_with_breaks(t, [])
    rss0, beta0 = rss_linear(y, X0)
    bic0 = bic_from_rss(rss0, n, X0.shape[1])
    best = {"breaks": [], "bic": bic0, "rss": rss0, "betas": beta0}

    for k in range(1, max_breaks + 1):
        for combo in itertools.combinations(candidates, k):
            ok = True
            last = 0
            for b in combo + (n,):
                if b - last < min_seg:
                    ok = False
                    break
                last = b
            if not ok:
                continue

            X = design_matrix_with_breaks(t, list(combo))
            rss, betas = rss_linear(y, X)
            bic = bic_from_rss(rss, n, X.shape[1])
            if bic < best["bic"]:
                best = {"breaks": list(combo), "bic": bic, "rss": rss, "betas": betas}

    return best

def chow_test_at_year(ts, year):
    y = ts.values
    t = np.arange(len(ts))
    d = (ts.index.year > year).astype(int)
    X = np.column_stack([np.ones(len(t)), t, d, d*t])
    full = sm.OLS(y, X).fit()

    Xr = sm.add_constant(t)
    restr = sm.OLS(y, Xr).fit()

    rss_r = restr.ssr
    rss_u = full.ssr
    k = X.shape[1] - Xr.shape[1]
    n = len(ts)

    f_stat = ((rss_r - rss_u) / k) / (rss_u / (n - X.shape[1]))
    from scipy.stats import f
    p_val = 1 - f.cdf(f_stat, k, n - X.shape[1])

    return {
        "F": float(f_stat),
        "p_value": float(p_val),
        "restricted_rss": float(rss_r),
        "unrestricted_rss": float(rss_u)
    }

def aic_search_arima(series, p_range=range(0,4), d_range=range(0,3), q_range=range(0,4)):
    best = {"aic": np.inf, "order": None, "fit": None}
    for p in p_range:
        for d in d_range:
            for q in q_range:
                try:
                    fit = ARIMA(series, order=(p,d,q)).fit()
                    if fit.aic < best["aic"]:
                        best = {"aic": fit.aic, "order": (p,d,q), "fit": fit}
                except:
                    continue
    return best


# 2. Output directories

base = "/home/karan/pyKaran/CarbonEmissions/Sheet4Plots/Sheet4PeiceWise"
os.makedirs(base, exist_ok=True)

dirs = {
    "plots": f"{base}",
    "csv": "/home/karan/pyKaran/CarbonEmissions/Sheet4csv"
}

for d in dirs.values():
    os.makedirs(d, exist_ok=True)

# 3. Analysis per time series
for (sub, sec), g in groups:
    name = f"{sub}_{sec}".replace(" ", "_")
    ts = pd.Series(g["Value"].values, index=g["Year"], name=name)

    # ---------------------------
    # Break detection
    # ---------------------------
    best = exhaustive_break_search(ts.values, max_breaks=3, min_seg=5)
    brk_idx = best["breaks"]
    brk_years = [ts.index[b].year for b in brk_idx]

    t = np.arange(len(ts))
    X = design_matrix_with_breaks(t, brk_idx)
    yhat = X @ best["betas"]

    # ---------------------------
    # Chow test at 2020
    # ---------------------------
    chow = chow_test_at_year(ts, 2019)

    # ---------------------------
    # Regime ARIMA models
    # ---------------------------
    all_breaks = brk_idx + [len(ts)]
    regimes = []
    start = 0
    for b in all_breaks:
        regs = ts.iloc[start:b]
        regimes.append((start, b, regs))
        start = b

    regime_rows = []
    for i, (s, e, segment) in enumerate(regimes, 1):
        best_arima = aic_search_arima(segment)
        regime_rows.append({
            "Regime": i,
            "Start_Year": int(segment.index[0].year),
            "End_Year": int(segment.index[-1].year),
            "Order": best_arima["order"],
            "AIC": best_arima["aic"]
        })

    # ---------------------------
    # Final forecast from last regime
    # ---------------------------
    last_seg = regimes[-1][2]
    best_last = aic_search_arima(last_seg)
    fit_last = best_last["fit"]

    steps = 10
    fc = fit_last.get_forecast(steps=steps)
    f_mean = fc.predicted_mean
    f_ci = fc.conf_int(alpha=0.05)

    future_years = pd.date_range(ts.index[-1] + pd.offsets.YearBegin(1),
                                 periods=steps, freq="YS")

    forecast_df = pd.DataFrame({
        "Year": future_years.year,
        "Forecast": f_mean.values,
        "Lower_95": f_ci.iloc[:,0].values,
        "Upper_95": f_ci.iloc[:,1].values
    })

    # ============================================================
    # PLOTS
    # ============================================================
    c_actual="#1f77b4"; c_piece="#2ca02c"; c_break="#9467bd"
    c_fc="#2ca02c"; c_ci="#7f7f7f"

    # --- Plot A: piecewise regression ---
    plt.figure(figsize=(12,6))
    plt.plot(ts.index, ts.values, color=c_actual, label="Actual", linewidth=2)
    plt.plot(ts.index, yhat, color=c_piece, label="Piecewise Fit", linewidth=2)
    for b in brk_idx:
        plt.axvline(ts.index[b], color=c_break, linestyle="--")
    plt.title(f"{name} – Piecewise Regression with Breaks")
    plt.xlabel("Year"); plt.ylabel("Value"); plt.grid(alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{dirs['plots']}/{name}_piecewise_fit.png", dpi=150)
    plt.close()

    # --- Plot B: Regimes + Forecast ---
    plt.figure(figsize=(12,6))
    plt.plot(ts.index, ts.values, color=c_actual, label="Actual", linewidth=2)

    prev = 0
    colors = ["#E8EEF7","#F7E8EF","#E8F7EF","#F7F3E8"]
    for i,b in enumerate(all_breaks):
        plt.axvspan(ts.index[prev], ts.index[b-1], color=colors[i%4], alpha=0.25)
        prev = b

    plt.plot(future_years, f_mean.values, color=c_fc, linewidth=2, label=f"Forecast (ARIMA{best_last['order']})")
    plt.fill_between(future_years, f_ci.iloc[:,0], f_ci.iloc[:,1],
                     color=c_ci, alpha=0.15, label="95% CI")

    for b in brk_idx:
        plt.axvline(ts.index[b], color=c_break, linestyle="--")

    plt.title(f"{name} – Regimes and 10-Year Forecast")
    plt.xlabel("Year"); plt.ylabel("Value"); plt.grid(alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{dirs['plots']}/{name}_regimes_forecast.png", dpi=150)
    plt.close()

    # ============================================================
    # CSV OUTPUTS
    # ============================================================
    pd.DataFrame([chow]).to_csv(f"{dirs['csv']}/{name}_chow_2020.csv", index=False)
    pd.DataFrame({"Break_Index":brk_idx,"Break_Year":brk_years}).to_csv(
        f"{dirs['csv']}/{name}_breaks.csv", index=False)
    pd.DataFrame(regime_rows).to_csv(f"{dirs['csv']}/{name}_regime_arima.csv", index=False)
    forecast_df.to_csv(f"{dirs['csv']}/{name}_10y_forecast.csv", index=False)



In [20]:
from collections import OrderedDict



#Legends Added
groups = df_long.groupby(["Substance", "Sector"])

# 1. Utility functions
def rss_linear(y, X):
    beta = np.linalg.lstsq(X, y, rcond=None)[0]
    resid = y - X @ beta
    return float(resid.T @ resid), beta

def bic_from_rss(rss, n, k):
    return n * np.log(rss / n) + k * np.log(n)

def design_matrix_with_breaks(t, breaks):
    X = np.ones((len(t), 1))
    X = np.column_stack([X, t])
    for b in breaks:
        hinge = np.maximum(0, t - b)
        X = np.column_stack([X, hinge])
    return X

def exhaustive_break_search(y, max_breaks=3, min_seg=5):
    n = len(y)
    t = np.arange(n)
    best = {"breaks": [], "bic": np.inf, "rss": None, "betas": None}
    candidates = list(range(min_seg, n - min_seg + 1))

    X0 = design_matrix_with_breaks(t, [])
    rss0, beta0 = rss_linear(y, X0)
    bic0 = bic_from_rss(rss0, n, X0.shape[1])
    best = {"breaks": [], "bic": bic0, "rss": rss0, "betas": beta0}

    for k in range(1, max_breaks + 1):
        for combo in itertools.combinations(candidates, k):
            ok = True
            last = 0
            for b in combo + (n,):
                if b - last < min_seg:
                    ok = False
                    break
                last = b
            if not ok:
                continue

            X = design_matrix_with_breaks(t, list(combo))
            rss, betas = rss_linear(y, X)
            bic = bic_from_rss(rss, n, X.shape[1])
            if bic < best["bic"]:
                best = {"breaks": list(combo), "bic": bic, "rss": rss, "betas": betas}
    return best

def chow_test_at_year(ts, year):
    y = ts.values
    t = np.arange(len(ts))
    d = (ts.index.year > year).astype(int)
    X = np.column_stack([np.ones(len(t)), t, d, d*t])
    full = sm.OLS(y, X).fit()

    Xr = sm.add_constant(t)
    restr = sm.OLS(y, Xr).fit()

    rss_r = restr.ssr
    rss_u = full.ssr
    k = X.shape[1] - Xr.shape[1]
    n = len(ts)

    f_stat = ((rss_r - rss_u) / k) / (rss_u / (n - X.shape[1]))
    from scipy.stats import f
    from collections import OrderedDict
    p_val = 1 - f.cdf(f_stat, k, n - X.shape[1])

    return {
        "F": float(f_stat),
        "p_value": float(p_val),
        "restricted_rss": float(rss_r),
        "unrestricted_rss": float(rss_u)
    }

def aic_search_arima(series, p_range=range(0,4), d_range=range(0,3), q_range=range(0,4)):
    best = {"aic": np.inf, "order": None, "fit": None}
    for p in p_range:
        for d in d_range:
            for q in q_range:
                try:
                    fit = ARIMA(series, order=(p,d,q)).fit()
                    if fit.aic < best["aic"]:
                        best = {"aic": fit.aic, "order": (p,d,q), "fit": fit}
                except:
                    continue
    return best


# ============================================================
# 2. Output directory setup
# ============================================================
base = "/home/karan/pyKaran/CarbonEmissions/Sheet4Plots/Sheet4PeiceWise"
os.makedirs(base, exist_ok=True)

dirs = {
    "plots": f"{base}",
    "csv": "/home/karan/pyKaran/CarbonEmissions/Sheet4csv"
}

# ============================================================
# 3. MAIN LOOP FOR ALL VARIABLES (WITH LEGENDS)
# ============================================================
for (sub, sec), g in groups:

    name = f"{sub}_{sec}".replace(" ", "_")
    ts = pd.Series(g["Value"].values, index=g["Year"], name=name)

    # Detect breakpoints
    best = exhaustive_break_search(ts.values, max_breaks=3, min_seg=5)
    brk_idx = best["breaks"]
    brk_years = [ts.index[b].year for b in brk_idx]

    t = np.arange(len(ts))
    X = design_matrix_with_breaks(t, brk_idx)
    yhat = X @ best["betas"]

    # Chow test at 2020
    chow = chow_test_at_year(ts, 2019)

    # Regime splitting
    all_breaks = brk_idx + [len(ts)]
    regimes = []
    start = 0
    for b in all_breaks:
        regimes.append(ts.iloc[start:b])
        start = b

    # ARIMA per regime
    regime_rows = []
    for i, segment in enumerate(regimes, 1):
        best_arima = aic_search_arima(segment)
        regime_rows.append({
            "Regime": i,
            "Start_Year": int(segment.index[0].year),
            "End_Year": int(segment.index[-1].year),
            "Order": best_arima["order"],
            "AIC": best_arima["aic"]
        })

    # Last regime forecast
    last_seg = regimes[-1]
    best_last = aic_search_arima(last_seg)
    fit_last = best_last["fit"]
    steps = 10
    fc = fit_last.get_forecast(steps=steps)
    f_mean = fc.predicted_mean
    f_ci = fc.conf_int(alpha=0.05)

    future_years = pd.date_range(ts.index[-1] + pd.offsets.YearBegin(1),
                                 periods=steps, freq="YS")

    forecast_df = pd.DataFrame({
        "Year": future_years.year,
        "Forecast": f_mean.values,
        "Lower_95": f_ci.iloc[:,0].values,
        "Upper_95": f_ci.iloc[:,1].values
    })


    # ============================================================
    # PLOTS WITH LEGENDS (INTEGRATED)
    # ============================================================

    c_actual="#1f77b4"; c_piece="#2ca02c"; c_break="#9467bd"
    c_fc="#2ca02c"; c_ci="#7f7f7f"

    # -------------------------- Plot A --------------------------
    plt.figure(figsize=(12,6))
    plt.plot(ts.index, ts.values, color=c_actual, linewidth=2, label="Actual")
    plt.plot(ts.index, yhat, color=c_piece, linewidth=2, label="Piecewise Fit")

    for j,b in enumerate(brk_idx):
        yr = ts.index[b].year
        lbl = f"Break {yr}" if j == 0 else None
        plt.axvline(ts.index[b], color=c_break, linestyle="--", alpha=0.8, label=lbl)

    plt.title(f"{name} – Piecewise Regression with Breaks")
    plt.xlabel("Year")
    plt.ylabel("Value")
    plt.grid(alpha=0.3)

    h,l = plt.gca().get_legend_handles_labels()
    pairs = [(lab, hdl) for hdl, lab in zip(h, l) if lab is not None]
    if pairs:
        od = OrderedDict(pairs)
        plt.legend(list(od.values()), list(od.keys()))
    else:
        plt.legend()

    plt.tight_layout()
    plt.savefig(f"{dirs['plots']}/{name}_piecewise_fit.png", dpi=150)
    plt.close()


    # -------------------------- Plot B --------------------------
    plt.figure(figsize=(12,6))
    plt.plot(ts.index, ts.values, color=c_actual, linewidth=2, label="Actual")

    prev = 0
    colors = ["#E8EEF7","#F7E8EF","#E8F7EF","#F7F3E8"]
    for i,b in enumerate(all_breaks):
        lbl = f"Regime {i+1}" if i < len(regimes) else None
        plt.axvspan(ts.index[prev], ts.index[b-1], color=colors[i%4], alpha=0.25, label=lbl)
        prev = b

    plt.plot(future_years, f_mean.values, label=f"Forecast (ARIMA{best_last['order']})",
             color=c_fc, linewidth=2)
    plt.fill_between(future_years, f_ci.iloc[:,0], f_ci.iloc[:,1],
                     color=c_ci, alpha=0.15, label="95% CI")

    for j,b in enumerate(brk_idx):
        yr = ts.index[b].year
        lbl = f"Break {yr}" if j == 0 else None
        plt.axvline(ts.index[b], color=c_break, linestyle="--", alpha=0.8, label=lbl)

    plt.title(f"{name} – Regimes and 10-Year Forecast")
    plt.xlabel("Year")
    plt.ylabel("Value")
    plt.grid(alpha=0.3)

    h,l = plt.gca().get_legend_handles_labels()
    plt.legend(OrderedDict(zip(l,h)).values(), OrderedDict(zip(l,h)).keys())

    plt.tight_layout()
    plt.savefig(f"{dirs['plots']}/{name}_regimes_forecast.png", dpi=150)
    plt.close()


    # ============================================================
    # CSV OUTPUTS
    # ============================================================
    pd.DataFrame([chow]).to_csv(f"{dirs['csv']}/{name}_chow_2020.csv", index=False)
    pd.DataFrame({"Break_Index":brk_idx,"Break_Year":brk_years}).to_csv(
        f"{dirs['csv']}/{name}_breaks.csv", index=False)
    pd.DataFrame(regime_rows).to_csv(f"{dirs['csv']}/{name}_regime_arima.csv", index=False)
    forecast_df.to_csv(f"{dirs['csv']}/{name}_10y_forecast.csv", index=False)

